In [5]:
import numpy as np
from netCDF4 import Dataset
import gzip
import tempfile
import shutil
%matplotlib inline
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap

ImportError: No module named netCDF4

In [2]:
import datetime

dates = []
for j in xrange(1,13): 
    d=1
    while True:
        try:
            date=datetime.date(2013,j,d).strftime('%Y%m%d')
            dates.append(date)
        except:
            break
        d += 1

MAB (Mean Absolute Bias) below 18 W/m²
and 85 % of absolute bias values below
20 W/m² (+ uncertainty of ground based
measurements) for monthly means.
Very low bias of 2.5 W/m² for monthly, daily
hourly means.
Higher bias values occur in the Alpine and
other mountainous regions, e. g. due to
uncertainties in area to point comparison and
snow coverage

In [3]:
latbounds =  [30,58]
lonbounds = [-20,37]

max_ind = 2
for j in xrange(0,max_ind):
    if np.mod(j,10) == 0:
        print "Processing file "+ str(j+1)
    dat_file = r'/Users/humphreys/Repositories/personal_calcs/Climate/ORD21164/DNIdm' + dates[j] + '0000002231000101MA.nc.gz'

    tmpfile = tempfile.NamedTemporaryFile()
    shutil.copyfileobj(gzip.open(dat_file), tmpfile)
    nc = Dataset(tmpfile.name, mode='r')

    SID = nc.variables['SID'][:].squeeze()
    
    if j == 0:
        
        lat = nc.variables['lat'][:]
        lon = nc.variables['lon'][:]

        latgridbounds = [np.argmin(abs(lat-i)) for i in latbounds]
        longridbounds = [np.argmin(abs(lon-i)) for i in lonbounds]
        lon = lon[longridbounds[0]:longridbounds[1]]
        lat = lat[latgridbounds[0]:latgridbounds[1]]
        
        lons,lats= np.meshgrid(lon,lat)
        SIDarray = np.zeros([max_ind,np.shape(lons)[0],np.shape(lons)[1]])
        
    SIDarray[j] = SID[latgridbounds[0]:latgridbounds[1],longridbounds[0]:longridbounds[1]]
    tmpfile.close()


Processing file 1


NameError: name 'tempfile' is not defined

In [4]:
# SIDmean = np.mean(SIDarray,axis=0)
runningAvgLength = 31
runningAvgFilter = np.ones((runningAvgLength,))/runningAvgLength

SIDmean = np.apply_along_axis(lambda m: np.convolve(m, runningAvgFilter, mode='valid'), axis=0, arr=SIDarray)

SIDarrayDif = SIDarray[((runningAvgLength-1)/2):-((runningAvgLength-1)/2)] - SIDmean

NameError: name 'SIDarray' is not defined

In [ ]:
fig=plt.figure()
ax = fig.add_axes([0.05,0.05,0.9,0.85])
# projmap = Basemap(projection='ortho',lat_0=45,lon_0=40,resolution='l')
projmap = Basemap(llcrnrlon=np.min(lon)-2,llcrnrlat=np.min(lat)-2,urcrnrlon=np.max(lon)+2,urcrnrlat=np.max(lat)+2,projection='mill',fix_aspect = True,resolution='l')
projmap.drawcoastlines()
projmap.drawcountries()
x,y = projmap(lons,lats)
temp = projmap.contourf(x,y,SIDarrayDif[-1])
cb = projmap.colorbar(temp,"bottom", size="5%", pad="2%")
plt.title('SID')
cb.set_label('SID')

fig=plt.figure()
ax = fig.add_axes([0.05,0.05,0.9,0.85])
# projmap = Basemap(projection='ortho',lat_0=45,lon_0=40,resolution='l')
projmap = Basemap(llcrnrlon=np.min(lon)-2,llcrnrlat=np.min(lat)-2,urcrnrlon=np.max(lon)+2,urcrnrlat=np.max(lat)+2,projection='mill',fix_aspect = True,resolution='l')
projmap.drawcoastlines()
projmap.drawcountries()
temp = projmap.contourf(x,y,SIDmean[-1])
cb = projmap.colorbar(temp,"bottom", size="5%", pad="2%")
plt.title('SID')
cb.set_label('SID')

Interesting detour - lets be correct about this - lat lon gives different distance between points depending on your vertical position (latitude)

In [ ]:
def haversine(lons, lats, reflat,reflon):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lons, lats, reflat, reflon = map(lambda x : (np.pi/180)*x, [lons, lats, reflat, reflon])

    # haversine formula 
    dlons = lons - reflon 
    dlats = lats - reflat 
    a = np.sin(dlats/2)**2 + np.cos(reflat) * np.cos(lats) * np.sin(dlons/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

def haversine(lons, lats, reflat,reflon):
    """
    Calculate the distance between points on earth
    """
    # convert decimal degrees to radians 
    lons, lats, reflat, reflon = map(lambda x : (np.pi/180)*x, [lons, lats, reflat, reflon])

    # haversine formula 
    dlons = lons - reflon 
    dlats = lats - reflat 
    a = np.sin(dlats/2)**2 + np.cos(reflat) * np.cos(lats) * np.sin(dlons/2)**2
    c = 2 * np.arcsin(np.sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [ ]:
lonscorrected = haversine(lons,lats,lat[0],lon[0])

In [ ]:
scipy.interpolate.interp2d(x, y, z, kind='cubic')

In [ ]:
import scipy
max_ind = np.shape(SIDarrayDif)[0]
for i in range(max_ind):
    if np.mod(j,10) == 0:
        print "Processing file "+ str(j+1)
        
    SID = SIDarrayDif[i]
    fftSID=np.fft.fft2(SID)
    cor=np.real(np.fft.ifft2(fftSID*np.conj(fftSID)))
    cor=cor/np.max(cor)
    if i == 0:
        cor_mean = cor
    else:
        cor_mean += cor

cor_mean = cor_mean/max_ind

plt.subplot(1,1,1)
plt.imshow(np.fft.fftshift(cor_mean))
plt.clim([-1,1])
plt.colorbar()